In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

df = pd.read_csv('train.csv')

In [42]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [43]:
df = df.dropna(subset=['Embarked'])
# Embarked_mapping = {'S':1, "C":2, "Q":3}
# df['Embarked'] = df['Embarked'].map(Embarked_mapping)
df['Embarked_C'], df['Embarked_Q'], df['Embarked_S'] = pd.get_dummies(df['Embarked'])['C'], pd.get_dummies(df['Embarked'])['Q'], pd.get_dummies(df['Embarked'])['S']
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0,0,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0,0,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,0,0


In [44]:
Sex_mapping = {'female':0, "male":1}
df['Sex'] = df['Sex'].map(Sex_mapping)
df['Sex']

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 889, dtype: int64

In [45]:
df['Title'] = df.Name.str.extract('([A-Za-z]+)\.', expand=False)
df_title = df['Title'].tolist()
titles = []
for i in range(len(df_title)):
    if df_title[i] not in titles:
        titles.append(df_title[i])
titles_age_mean = df.groupby('Title')['Age'].median()
titles_age_mean.tolist()
title_uniq = df['Title'].unique().tolist()
title_uniq.sort()
titles_age_mapping = {}
for i in range(len(titles_age_mean)):
    titles_age_mapping[title_uniq[i]]= titles_age_mean[i]
titles_age_mapping

{'Capt': 70.0,
 'Col': 58.0,
 'Countess': 33.0,
 'Don': 40.0,
 'Dr': 46.5,
 'Jonkheer': 38.0,
 'Lady': 48.0,
 'Major': 48.5,
 'Master': 3.5,
 'Miss': 21.0,
 'Mlle': 24.0,
 'Mme': 24.0,
 'Mr': 30.0,
 'Mrs': 35.0,
 'Ms': 28.0,
 'Rev': 46.5,
 'Sir': 49.0}

In [46]:
df['Age'] = df['Age'] .fillna(df['Title'].map(titles_age_mapping))
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Embarked_C       0
Embarked_Q       0
Embarked_S       0
Title            0
dtype: int64

In [47]:
from sklearn.preprocessing import StandardScaler

bin=[0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80]
df['Age_cut'] = pd.cut(df['Age'],bin, labels=[0,1,2,3,4,5,6,7,8,9])
pd.value_counts(df['Age_cut'])
scaler = StandardScaler()
age_scale_param = scaler.fit(df['Age'].values.reshape(-1, 1))
df['Age_scaled'] = scaler.fit_transform(df['Age'].values.reshape(-1, 1), age_scale_param)
fare_scale_param = scaler.fit(df['Fare'].values.reshape(-1, 1))
df['Fare_scaled'] = scaler.fit_transform(df['Fare'].values.reshape(-1, 1), age_scale_param)

In [48]:
from sklearn.model_selection import train_test_split

X, y = pd.DataFrame(), pd.DataFrame()
#X['Pclass'], X['Sex'], X['Age_cut'], X['Embarked'] = df['Pclass'], df['Sex'], df['Age_cut'], df['Embarked'] 
X['Fare_scaled'], X['Pclass'], X['Sex'], X['Age_cut'], X['Embarked_C'], X['Embarked_Q'], X['Embarked_S'] = df['Fare_scaled'], df['Pclass'], df['Sex'], df['Age_cut'], df['Embarked_C'], df['Embarked_Q'], df['Embarked_S']
 
y['Survived'] = df['Survived']
X, y = X.values, y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=1)

In [49]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import validation_curve

# param_range = np.arange(1, 80, 2)

# train_scores, test_scores = validation_curve(RandomForestClassifier(), 
#                                              X=X_train, 
#                                              y=y_train, 
#                                              param_name="n_estimators", 
#                                              param_range=param_range,
#                                              cv=2, 
#                                              scoring="accuracy", 
#                                              n_jobs=-1)
# train_mean = np.mean(train_scores, axis=1)
# train_std = np.std(train_scores, axis=1)
# test_mean = np.mean(test_scores, axis=1)
# test_std = np.std(test_scores, axis=1)

# plt.plot(param_range, train_mean, color='blue', marker='o', markersize=5, label='訓練集準確度')
# plt.fill_between(param_range, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

# plt.plot(param_range, test_mean, color='green', linestyle='--', marker='s', markersize=5, label='訓練集準確度')
# plt.fill_between(param_range, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')

# plt.grid()
# plt.xlabel('Tree Number')
# plt.ylabel('Accuracy')
# plt.ylim(0.7,0.9)
# plt.show()

In [50]:
# from sklearn.decomposition import KernelPCA

# kpca = KernelPCA(n_components=2, kernel='rbf')
# X_kpca = kpca.fit_transform(X)
# X_train_kpca = kpca.fit_transform(X_train)
# X_test_kpca = kpca.fit_transform(X_test)


In [51]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
pipe_svc = make_pipeline(SVC(random_state=0))
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=2,
                  n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
print(gs.best_params_)

0.7941643891082093
{'svc__C': 10.0, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}


In [52]:
# forest = RandomForestClassifier(random_state=1)
# PRF=[{'n_estimators':[10,100],'max_depth':[3,6],'criterion':['gini','entropy']}]
# GSRF=GridSearchCV(estimator=forest, param_grid=PRF, scoring='accuracy',cv=2)
# GSRF.fit(X, y)
# print(GSRF.best_score_)
# print(GSRF.best_params_)

In [53]:
import os
os.getcwd()

final_test_df = pd.read_csv('test.csv')
#final_test_df['Embarked'] = final_test_df['Embarked'].map(Embarked_mapping)
final_test_df['Sex'] = final_test_df['Sex'].map(Sex_mapping)
final_test_df['Age'] = final_test_df['Age'] .fillna(df['Title'].map(titles_age_mapping))
final_test_df['Fare'] = final_test_df['Fare'].fillna(df['Fare'].mean())
final_test_df['Age_cut'] = pd.cut(final_test_df['Age'],bin, labels=[0,1,2,3,4,5,6,7,8,9])
final_test_df['Embarked_C'], final_test_df['Embarked_Q'], final_test_df['Embarked_S'] = pd.get_dummies(final_test_df['Embarked'])['C'], pd.get_dummies(final_test_df['Embarked'])['Q'], pd.get_dummies(final_test_df['Embarked'])['S']

age_scale_param = scaler.fit(final_test_df['Age'].values.reshape(-1, 1))
final_test_df['Age_scaled'] = scaler.fit_transform(final_test_df['Age'].values.reshape(-1, 1), age_scale_param)

fare_scale_param = scaler.fit(final_test_df['Fare'].values.reshape(-1, 1))
final_test_df['Fare_scaled'] = scaler.fit_transform(final_test_df['Fare'].values.reshape(-1, 1), age_scale_param)

final = pd.DataFrame()
#final['Pclass'], final['Sex'], final['Age_cut'], final['Embarked'] = final_test_df['Pclass'], final_test_df['Sex'], final_test_df['Age_cut'], final_test_df['Embarked'] 
final['Fare_scaled'], final['Pclass'],  final['Sex'], final['Age_cut'], final['Embarked_C'], final['Embarked_Q'], final['Embarked_S']  = final_test_df['Fare_scaled'], final_test_df['Pclass'], final_test_df['Sex'], final_test_df['Age_cut'], final_test_df['Embarked_C'], final_test_df['Embarked_Q'], final_test_df['Embarked_S'] 

# final_kpca = kpca.fit_transform(final)
submission = pd.DataFrame()
submission['PassengerId'] = final_test_df['PassengerId']
submission['Survived'] = gs.predict(final)
submission.to_csv('submission.csv', index=False)